In [1]:
from map import Map
from algoviz import AlgoViz
from algoviz.svg import SVGView, Circle, Rect, Text
from algoviz.gui import GUI, Button, Label, TextInput
import random, math, time
from random import randrange
from player import Player
from enemy import Enemy


running = True
currentroom = 0
gegner = []
rooms = []

l=1000  # spielfeldgröße
view = SVGView(l,l)


def start_game():
    
    
    
    global currentroom, running, gegner, rooms, l, view
    
    feld = Rect(0,0,l,l,view)
    feld.set_fill("white")
    
    
    rooms = create_game()
    
    pl = Player(l // 2, l // 2, 15, view)
    
    
    

    rooms[currentroom].zeichnen(view, l)
    
    while running == True:
        jetzt = time.time()
        
        pl.change_speed()
        pl.check_wall_collision(rooms[currentroom], l)
        pl.bewegen()
        pl.attack(jetzt, gegner)

        for i in range(len(gegner) - 1, -1, -1):
            if gegner[i].get_hp() <= 0:
                gegner[i]._kreis.set_fill("white")
                gegner.pop(i)
            else:
                gegner[i].move(pl.get_x(), pl.get_y(), jetzt)
                if gegner[i].hits_player(pl.get_x(), pl.get_y()):
                    pl.is_hit(jetzt, 50)
                    if pl.get_hp() <= 0:
                        running = False
        
        #for i in items:
            #i.is_collected(pl.get_x(), pl.get_y())
        
        # move_room()
        # pl.move_to(0,0)
        # pl.tofront
        #enemy.move(pl.getx, pl.gety

        buttons(view.pressed_key(), pl)           
        view.sync()


In [2]:
def create_game():
    rooms = []

    lobby = Map(8,8)
    lobby.create_lobby()
    rooms.append(lobby)
    
    for i in range(randrange(3,5)):
        level = Map(15,15)
        level.create_maze(2)
        rooms.append(level)

    end = Map(15,15)
    end.create_end()
    rooms.append(end)

    return rooms

In [3]:
def buttons(key, pl):
    global running
    if key == "p":
        running = False

    if key == "l":
        
        move_room()
        pl.move_to(l//2,l//2)
        pl.to_front()
        pl.attack_circle.to_front()

In [4]:
def move_room():
    global currentroom, running, gegner, rooms, l, view
    rect = Rect(0,0,l,l,view)
    rect.set_fill("white")
    if currentroom == len(rooms)-1:
        running = False
        print("spiel vorbei")
        
    else:
        currentroom += 1
        rooms[currentroom].zeichnen(view, l)

        gegner = []
        items = []
        
        feld = rooms[currentroom]
        
        mazew = feld.get_length()
        mazeh = feld.get_height()
        tile = l//mazew
        for i in range(mazew):
             for j in range(mazeh):
                 if feld.layout[i][j] == 3:
                     print("objekt")
                 if feld.layout[i][j] == 2:
                     gegner.append(Enemy(j*(l//mazew) + (tile//2),i*(l//mazeh) + (tile//2), 15, view, feld))
                     print("gegner erstellt")
                 
                

In [ ]:
start_game()

gegner erstellt
gegner erstellt
